In [ ]:
!pip install summac
!pip install --upgrade transformers huggingface_hub
import nltk
nltk.download('punkt_tab')

INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 531.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from summac.model_summac import SummaCZS, SummaCConv
import pandas as pd

model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cuda") # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cuda", start_file="default", agg="mean")

csv_filename = "input.csv"
df = pd.read_csv(csv_filename)
model_name = [
    'reference',
    'access',
    'keep_it_simple',
    'swipe_v5',
    'swipe_v5_clean',
    'gpt3',
    'bartl_wikilarge',]

for name in model_name:
    df["SummaCZS_"+name] = ""
    df["SummaCConv_"+name] = ""
"""
    'r_content',
    's_content',
    'candidate_access',
    'candidate_keep_it_simple',
    'candidate_swipe_v5',
    'candidate_swipe_v5_clean',
    'candidate_gpt3',
    'candidate_bartl_wikilarge',
"""

for index, row in df.iterrows():
    document = row['r_content']
    for name in model_name:
      if name == 'reference':
        summary = row['s_content']
      else:
        summary = row['candidate_'+name]
      df.at[index, "SummaCZS_"+name] = model_zs.score([document], [summary])["scores"][0]
      df.at[index, "SummaCConv_"+name] = model_conv.score([document], [summary])["scores"][0]

new_csv_filename = "summac.csv"
df.to_csv(new_csv_filename, index=False)

print(f"CSV has been updated and salved as: {new_csv_filename}")

/usr/local/lib/python3.11/dist-packages/summac/model_summac.py:250: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  print(self.load_state_dict(torch.load(start_file)))


<All keys matched successfully>
CSV 文件已更新并保存为: updated_model_responses.csv
